In [1]:
WEIGHTS_V = 5
CLASSES = 32

In [2]:
import os

FOLDER_PATH = os.path.join("Letters Recognition", "collected_images_v10")

In [3]:
import pandas as pd

train_labels = pd.read_csv(os.path.join(FOLDER_PATH, "train.csv"))
val_labels = pd.read_csv(os.path.join(FOLDER_PATH, "val.csv"))
test_labels = pd.read_csv(os.path.join(FOLDER_PATH, "test.csv"))

# train_labels = train_labels.append(test_labels)

train_labels

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,label
0,0.682235,0.653362,0.646337,0.661332,0.623815,0.677743,0.610539,0.696525,0.595573,0.708032,...,0.762554,0.684893,0.702454,0.678845,0.727027,0.674011,0.741386,0.668766,0.753137,28
1,0.262694,0.756871,0.309232,0.736183,0.341331,0.704608,0.356621,0.680913,0.359279,0.659747,...,0.713276,0.225852,0.676545,0.271111,0.685988,0.274742,0.709457,0.263442,0.715129,13
2,0.180026,0.431118,0.207235,0.414686,0.220569,0.391116,0.214714,0.370842,0.209489,0.354980,...,0.336195,0.153120,0.375646,0.158347,0.360131,0.170939,0.353727,0.185943,0.351210,25
3,0.737893,0.755567,0.703672,0.738311,0.683265,0.711524,0.676846,0.688623,0.684613,0.672562,...,0.626708,0.773923,0.712850,0.776373,0.689630,0.775371,0.673663,0.772285,0.657043,11
4,0.827406,0.678540,0.799623,0.645789,0.756141,0.619003,0.711802,0.610898,0.679800,0.613926,...,0.648147,0.746763,0.636778,0.694240,0.655960,0.707613,0.668070,0.725981,0.667468,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,0.317585,0.804309,0.350308,0.790970,0.372615,0.767727,0.378036,0.743254,0.389498,0.724128,...,0.675232,0.290349,0.743843,0.280373,0.724272,0.274581,0.710517,0.270305,0.696106,12
1568,0.213967,0.739656,0.238297,0.733154,0.256640,0.720546,0.267153,0.712424,0.270690,0.707738,...,0.735184,0.179048,0.687234,0.200036,0.698556,0.209246,0.716797,0.213815,0.729033,15
1569,0.763975,0.719866,0.741540,0.703115,0.728158,0.679427,0.726449,0.662741,0.721726,0.646599,...,0.700242,0.806987,0.663720,0.789135,0.670120,0.776311,0.684017,0.768498,0.694748,16
1570,0.253361,0.730760,0.290526,0.717379,0.323798,0.691682,0.347546,0.673649,0.374163,0.665142,...,0.683931,0.218914,0.664489,0.234541,0.670323,0.241096,0.685243,0.236197,0.685319,14


In [4]:
import torch
from torch_geometric.data import Data
import pandas as pd

edges = [
         [3, 4], [0, 5], [17, 18], [0, 17], [13, 14], [13, 17], [18, 19], [5, 6], [5, 9], [14, 15],
         [0, 1], [9, 10], [1, 2], [9, 13], [10, 11], [19, 20], [6, 7], [15, 16], [2, 3], [11, 12], [7, 8],
]

def create_graph(edge_index, kp, y):
    node_features = list()
    for i in range(0, len(kp), 2):
        node_features.append([kp[i], kp[i+1]])
    
    edge_index = torch.tensor(edges, dtype=torch.long)    
    x = torch.tensor(node_features, dtype=torch.float)
    # return Data(x=x, edge_index=edge_index.t().contiguous(), y=y).pin_memory().to(device, non_blocking=True)
    return Data(x=x, edge_index=edge_index.t().contiguous(), y=y)

def build_graph_data(df):
    data = list()
    for i in range(len(df)):
        graph = create_graph(None, df.iloc[i][:-1], int(df.iloc[i][-1]))
        data.append(graph)
    
    return data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data = build_graph_data(train_labels)
val_data = build_graph_data(val_labels)
test_data = build_graph_data(test_labels)

### Create DataLoaders

In [5]:
from torch_geometric.loader import DataLoader

batch_size = 4096

train_dl = DataLoader(train_data, batch_size=batch_size)
val_dl = DataLoader(val_data, batch_size=batch_size)
test_dl = DataLoader(test_data, batch_size=batch_size)

In [6]:
from torch.nn import Linear, ReLU
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, Sequential, global_mean_pool, global_max_pool

class GCN(torch.nn.Module):
    def __init__(self, num_features, classes, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(num_features, 64)
        self.conv2 = GCNConv(64, 128)
        self.conv3 = GCNConv(128, 128)
        self.conv4 = GCNConv(128, 64)
        self.lin = Linear(64, classes)

    def forward(self, x, edge_index, batchh):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        x = x.relu()
        
        # 2. Readout layer
        x = global_max_pool(x, batchh)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        # x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(num_features=train_data[0].num_features, classes=CLASSES, hidden_channels=64)
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GCN(
  (conv1): GCNConv(2, 64)
  (conv2): GCNConv(64, 128)
  (conv3): GCNConv(128, 128)
  (conv4): GCNConv(128, 64)
  (lin): Linear(in_features=64, out_features=32, bias=True)
)
Number of parameters:  35360


## New Training Process

In [7]:
import copy
from torch.optim.lr_scheduler import StepLR
from utils.rgb_frames.trainers import GraphTrainer
from torch.autograd import Variable

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = GCN(num_features=train_data[0].num_features, classes=CLASSES, hidden_channels=64).to(device, non_blocking=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_func = torch.nn.CrossEntropyLoss()
lr_scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

trainer = GraphTrainer(model=model, opt=optimizer, loss_func=loss_func, lr_scheduler=None)

params_train = {
    "train_dl": train_dl,
    "val_dl": val_dl,
    "start_epoch": 1,
    "end_epoch": 1000,
    "path_to_weights": "weights/weights_img_gcn" + "_c" + str(CLASSES) + "_v" + str(WEIGHTS_V) + ".tar",
    "path_to_checkpoint": "weights/checkpoint_img_gcn" + "_c" + str(CLASSES) + "_v" + str(WEIGHTS_V) + ".tar",
    "device": device,
    "progress_bar": False
}

In [8]:
WEIGHTS_V += 1
trained_model, loss_history, metric_history = trainer.train_val(**params_train)

Epoch 1/1000, current lr=0.01
Copied best model weights!
train loss: 0.002205, train_accuracy: 3.12, val_loss: 0.008785, val_accuracy: 3.05
Checkpoint saved
----------
Epoch 2/1000, current lr=0.01
Copied best model weights!
train loss: 0.002196, train_accuracy: 3.88, val_loss: 0.008735, val_accuracy: 3.81
Checkpoint saved
----------
Epoch 3/1000, current lr=0.01
Copied best model weights!
train loss: 0.002186, train_accuracy: 3.75, val_loss: 0.008648, val_accuracy: 4.06
Checkpoint saved
----------
Epoch 4/1000, current lr=0.01
Copied best model weights!
train loss: 0.002164, train_accuracy: 4.64, val_loss: 0.008508, val_accuracy: 4.31
Checkpoint saved
----------
Epoch 5/1000, current lr=0.01
Copied best model weights!
train loss: 0.002131, train_accuracy: 4.77, val_loss: 0.008342, val_accuracy: 3.81
Checkpoint saved
----------
Epoch 6/1000, current lr=0.01
Copied best model weights!
train loss: 0.002103, train_accuracy: 5.41, val_loss: 0.008241, val_accuracy: 5.84
Checkpoint saved
---